# python launcher

In [1]:
import sys
if 'ipykernel_launcher.py' not in sys.argv[0]:
    manual = False
    scenario = sys.argv[1]
else:
    manual = True
    scenario = 'intermodal'
    %matplotlib inline

In [2]:
#%matplotlib inline

In [3]:
import pandas as pd

# import 

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np

work_path = r'../../../'
quetzal_path = work_path + r'quetzal_santo_domingo/'

import sys
sys.path.insert(0, '../../../quetzal/')

from quetzal.model import stepmodel
from quetzal.io import display

SQLalchemy is not installed. No support for SQL output.


In [5]:
distributed =  stepmodel.read_zip(quetzal_path + 'model/transport/distribution.zip')

zones: 100%|██████████| 29/29 [00:00<00:00, 40.95it/s]          


In [6]:
car =  stepmodel.read_zip(quetzal_path + 'model/transport/car_pathfinder.zip')
pt = stepmodel.read_zip(quetzal_path + 'model/'+ scenario +'/pt_pathfinder.zip')

zones: 100%|██████████| 26/26 [00:03<00:00,  6.62it/s]       


In [7]:
sm = pt.copy()
sm.car_los = car.car_los

sm.road_nodes = car.road_nodes
sm.road_links = car.road_links

In [8]:
sm.volumes = distributed.volumes

# prep

In [9]:
self = sm
for value in ['length']:
    value_dict = self.road_links[value].to_dict()
    self.car_los[value] = self.car_los['link_path'].apply(
        lambda p: sum([value_dict[l] for l in p])
    ) 

# Utility parameters

In [10]:
parameter_frame = pd.read_excel(quetzal_path + r'inputs/parametros.xlsx').set_index(['category','parameter'])
for c in parameter_frame.columns:
    parent = parameter_frame[c][('general', 'parent')]
    try:
        parent = parent.iloc[0]
    except AttributeError:# 'str' object has no attribute 'iloc'
        pass
    parameter_frame[c] = parameter_frame[c].fillna(parameter_frame[parent])
var = parameter_frame[scenario]

year = int(var['general', 'year'])

In [11]:
var

category                 parameter          
PARAMETROS DE ESCENARIO  NaN                                  NaN
general                  parent                 ambicioso_sitp_30
                         year                                2030
                         freeze_modal_split                 false
                         freeze_distribution                false
                                                      ...        
                         footpath_speed                       3.4
                         short_leg_speed                      4.4
                         long_leg_speed                        12
                         n_ntlegs                              10
                         ntleg_threshold                      300
Name: intermodal, Length: 105, dtype: object

# volumes

In [12]:
if var['general','freeze_distribution'].iloc[0].lower() == 'true':
    sm.volumes['car'] = sm.volumes['survey_car']
    sm.volumes['nocar'] = sm.volumes['survey_pt'] + sm.volumes['survey_walk']
    delta = year - 2018
    for segment in ['car', 'nocar']:
        sm.volumes[segment] = sm.volumes[segment]  * np.power(sm.volumes['growth'], delta)
else :
    sm.volumes[['car', 'nocar']] = sm.volumes[['car' + str(year), 'nocar' + str(year)]]

In [13]:
car_speed = float(var['speed']['car'])
gas_per_km = float(var['general']['gas_per_km']) # gal US per km (1 US gal= 3.7L)
gas_price = float(var['general']['gas_price'])# DOP per gal US
price_per_m =  gas_per_km * gas_price / 1000

# Fares

## PT fares

In [14]:
# Cálculo de tarifas según número de transferencias permitidas

for t in [0,1,2,3,4,5,'n']:
    name = 'f' + str(t)

    if t!='n':
        var['transfers', name] = t
    else:
        var['transfers', name] = 10

    price = min(var['base']['fx'] + (var['transfers'][name]*var['add']['fx']), var['max']['fx'])
    var['price', name] = price

# OJO: Solo para escenarios tarifarios -> OPRET se integra, respetando gratuidad entre metro-teleferico

var['price','opret'] = var['base']['fx']

In [15]:
fare_attributes = pd.DataFrame(
    {
    'price': var['price'],
    'transfers': var['transfers']
    }
)
fare_attributes.index.name = 'fare_id'
fare_attributes.reset_index(inplace=True)
fare_attributes[['price', 'transfers']] = fare_attributes[['price', 'transfers']].astype(float)
fare_attributes.loc[fare_attributes['fare_id'] == 'integrated', 'transfers'] = np.inf

# CHANGE TO TRUE TO TEST FARES | FALSE FOR THE FORMER BEHAVIOUR
add_custom_fare_attributes = False
#TODO change that so it is automated somehow
if add_custom_fare_attributes:
    fare_attributes = fare_attributes.append(pd.Series({'fare_id': 'direct', 'price': 25, 'transfers': 0}), ignore_index=True)
    fare_attributes = fare_attributes.append(pd.Series({'fare_id': 'opret', 'price': 20, 'transfers': np.inf}), ignore_index=True)
fare_attributes

,fare_id,price,transfers
0,bus,15.0,0.0
1,concho,35.0,0.0
2,express_bus,30.0,0.0
3,f0,35.0,0.0
4,f1,50.0,1.0
5,f2,65.0,2.0
6,f3,80.0,3.0
7,f4,80.0,4.0
8,f5,80.0,5.0
9,fn,80.0,10.0


In [16]:
left = left = var['fare_id'].apply(lambda s: [item.strip() for item in  s.split(',')]).explode().reset_index()
left.columns = ['route_type', 'fare_id']

#TODO change that so it is automated somehow
if add_custom_fare_attributes:
    left = pd.concat(
        [
            left, 
            left.loc[left['fare_id']=='integrated'].replace('integrated', 'direct'),
            left.loc[left['route_type'].isin(['subway'])].replace('integrated', 'opret')
        ]
    )


In [17]:
right = pt.lines[['route_type', 'route_id']]

In [18]:
fare_rules = pd.merge(left, right)[['fare_id', 'route_id']]
fare_rules.head()

,fare_id,route_id
0,concho,CP_C100_1
1,concho,CP_C100_2
2,concho,CP_C101E_1
3,concho,CP_C101E_2
4,concho,CP_C102_1


In [19]:
fare_attributes['agency_id'] = 'intrant'

In [20]:
sm.links['agency_id'] = 'intrant'
sm.nodes['zone_id'] = 0

In [24]:
sm.fare_rules = fare_rules
sm.fare_attributes = fare_attributes.copy()

opret_routes = {'m1', 'm2', 'm2_completo', 'tele_este'}
sm.compute_arod_list(tap_free_networks=[opret_routes])


{'tele_este': {'tele_este', 'm2_completo', 'm1', 'm2'}, 'm2_completo': {'tele_este', 'm2_completo', 'm1', 'm2'}, 'm1': {'tele_este', 'm2_completo', 'm1', 'm2'}, 'm2': {'tele_este', 'm2_completo', 'm1', 'm2'}}


In [25]:
sm.fare_attributes

,fare_id,price,transfers,agency_id
0,bus,15.0,0.0,intrant
1,concho,35.0,0.0,intrant
2,express_bus,30.0,0.0,intrant
3,f0,35.0,0.0,intrant
4,f1,50.0,1.0,intrant
5,f2,65.0,2.0,intrant
6,f3,80.0,3.0,intrant
7,f4,80.0,4.0,intrant
8,f5,80.0,5.0,intrant
9,fn,80.0,10.0,intrant


In [26]:
# FORMER FARES
forbidden = {
    'f0': {'f1', 'f2', 'f3', 'f4', 'f5'},
    'f1': {'f0', 'f2', 'f3', 'f4', 'f5'},
    'f2': {'f0', 'f1', 'f3', 'f4', 'f5'},
    'f3': {'f0', 'f1', 'f2', 'f4', 'f5'},
    'f4': {'f0', 'f1', 'f2', 'f3', 'f5'},
    'f5': {'f0', 'f1', 'f2', 'f3', 'f4'},
} # fares that are irrelevant when consecutive
sm.compute_route_fares(
    irrelevant_consecutive_fares=forbidden, 
    consecutive=False, 
)
sm.pt_los['price'] = sm.pt_los['route_fares'].apply(lambda breakdown: sum(breakdown.values()))

In [ ]:
if manual:
    sm.pt_los.loc[~sm.pt_los.all_walk][['arod_list','route_fares','price']].to_excel(r'../../model/latest/test_'+scenario+'.xlsx')

## car fares

In [ ]:
# length
self = sm
sm.road_links['toll'] = 0

In [ ]:
self.car_los['price'] = self.car_los['length'] * price_per_m #+ self.car_los['toll']

# Time

## walk_time

In [ ]:
short_leg_speed = float(var['general']['short_leg_speed'])
long_leg_speed = float(var['general']['long_leg_speed'])
threshold= float(var['general']['ntleg_threshold'])
footpath_speed= float(var['general']['footpath_speed'])
n_ntlegs = int(var['general']['n_ntlegs'])

In [ ]:
sm.footpaths['time'] = sm.footpaths['length']  / 1000 * 3600 / footpath_speed
#sm.zone_to_transit['time'] = sm.zone_to_transit['distance']  / 1000 * 3600 / short_leg_speed
#sm.zone_to_road['time'] = sm.zone_to_road['distance']  / 1000 * 3600 / short_leg_speed

In [ ]:
sm.preparation_ntlegs(
    n_ntlegs=n_ntlegs,
    short_leg_speed=short_leg_speed,
    long_leg_speed=long_leg_speed,
    zone_to_road=True,
)

## car_time

In [ ]:
sm.road_links['speed'] = car_speed
sm.road_links['time'] = sm.road_links['length'] / 1000 * 3600 / sm.road_links['speed']

In [ ]:
sm.zone_to_road  = car.zone_to_road

In [ ]:
sm.analysis_car_time()
sm.analysis_car_length()

## pt_time

In [ ]:
from shapely import geometry
def straight_length(ls):
    return geometry.LineString([list(ls.coords)[0],  list(ls.coords)[-1]]).length

In [ ]:
sm.links['length'] = sm.links['geometry'].apply(straight_length)

In [ ]:
if False:
    sm.links['speed'] = link_speed
    sm.links.loc[sm.links['route_type'] == 'subway', 'speed'] = metro_speed
    sm.links['time'] =  sm.links['length'] / 1000 * 3600 / sm.links['speed']

In [ ]:
sm.analysis_pt_time(boarding_time=0)
sm.analysis_pt_length()

# transfers

In [ ]:
sm.car_los['ntransfers'] = 0
sm.pt_los['ntransfers'] = sm.pt_los['alighting_links'].apply(len)

# route_types

In [ ]:
mode_hierarchy = ['car', 'subway', 'tram',  'gondola', 'express_bus', 'bus', 'minibus', 'concho', 'walk']

In [ ]:
sm.los = sm.pt_los
sm.analysis_pt_route_type(hierarchy=mode_hierarchy)
sm.analysis_car_route_type()

In [ ]:
sm.lighten_pt_los()
sm.analysis_pt_los()
sm.lighten_car_los()
sm.analysis_car_los()
sm.los = pd.concat([sm.car_los, sm.pt_los]).reset_index(drop=True)

In [ ]:
#sm.los.dropna(axis=1, inplace=True)

# UTILITY

In [ ]:
loc = (sm.los['route_type'] == 'walk') & (sm.los['time'] > 1800)
sm.los.loc[loc, 'price'] = 100

In [ ]:
route_types = set(sm.links['route_type'].unique()).union({'car', 'walk'})

In [ ]:
# Average
sm.preparation_logit(
    mode=1, 
    pt_mode=0.5, 
    pt_path=0.2, 
    time=-1 / 3600,
    price= -1 / 50,
    transfers=-0.2,
    segments=['car', 'nocar'],
)

In [ ]:
sm.mode_utility[['car', 'nocar']] = 0
sm.mode_utility.loc[['car'], ['nocar', 'car']] = [-100, 0.5] # [-100, 0] 
sm.mode_utility.loc[['minibus', 'concho'], ['nocar', 'car']] = 0

if var['general','freeze_modal_split'].iloc[0].lower() == 'true':
    sm.mode_utility.loc[['car'], ['nocar', 'car']] = [-100, 100] # [-100, 0] 

In [ ]:
sm.mode_utility -= sm.mode_utility.max()

In [ ]:
sm.logit_scales
sm.mode_utility

In [ ]:
sm.utility_values.loc['time', 'car'] = sm.utility_values.loc['time', 'nocar'] * 2

# here

In [ ]:
sm.los.reset_index(drop=True, inplace=True)

In [ ]:
sm.los['index'] = sm.los.index

# utility

In [ ]:
sm.analysis_mode_utility(how='min', segment='car')
sm.analysis_mode_utility(how='min', segment='nocar')

In [ ]:
sm.initialize_logit()

In [ ]:
sm.segments = ['car', 'nocar']

In [ ]:
sm.logit_scales['nocar'] = sm.logit_scales['car']

In [ ]:
sm.logit_scales

In [ ]:
sm.step_logit(nchunks=5)

In [ ]:
sm.od_probabilities = sm.probabilities.set_index(
    ['origin', 'destination', 'segment']
).unstack('segment').swaplevel(axis='columns')
sm.od_utilities = sm.utilities.set_index(
    ['origin', 'destination', 'segment']
).unstack('segment').swaplevel(axis='columns')

In [ ]:
sm.od_probabilities

In [ ]:
t = sm.los['path'].apply(tuple)

# Control

In [ ]:
if manual:
    %matplotlib inline

In [ ]:
sm.volumes = distributed.volumes.copy()
#sm.volumes.loc[sm.volumes['origin'] == sm.volumes['destination'], ['car', 'nocar']] = 0 

In [ ]:
sm.od = sm.volumes.set_index(['origin', 'destination'])[['car', 'nocar']] 

In [ ]:
sm.probabilities

In [ ]:
a = sm.od_probabilities.copy()
a.columns = pd.MultiIndex.from_tuples(a.columns)

In [ ]:
voy = a['car'].apply(lambda c: c * sm.od['car']) + a['nocar'].apply(lambda c: c * sm.od['nocar']) 

In [ ]:
voykil = voy.apply(
    lambda c: c * distributed.euclidean.set_index(
    ['origin', 'destination']).sort_index()['km']
)


s = voy.sum()[sorted(set(sm.mode_nests.index) - set(sm.mode_nests['root']))]
plot = s.plot(
    kind='pie', 
    autopct='%1.0f%%',
    shadow=False,
    startangle=90, figsize=[5, 5])
plot.set_aspect('equal')

In [ ]:
s = voykil.sum()[sorted(set(sm.mode_nests.index) - set(sm.mode_nests['root']))]
plot = s.plot(
    kind='pie', 
    autopct='%1.0f%%',
    shadow=False,
    startangle=90, figsize=[5, 5])
plot.set_aspect('equal')

In [ ]:
s = voykil.sum()[sorted(set(sm.mode_nests.index) - set(sm.mode_nests['root']))]

# Assignment

In [ ]:
sm.car_los['path'] = sm.car_los['path'].apply(tuple)
sm.pt_los['path'] = sm.pt_los['path'].apply(tuple)
sm.los['path'] = sm.los['path'].apply(tuple)

## PT

# segmented assignment

In [ ]:
sm.step_assignment(boarding_links=True, alightings=True, transfers=True, road=True, segmented=False)

In [ ]:
sm.road_links.loc[53982]

In [ ]:
links = sm.links#.loc[sm.links['route_type'] == 'subway']
links = sm.links.loc[sm.links['volume'] > 0]
links['label'] = ''

In [ ]:
links['volume'].max()

In [ ]:
sm.road_links['volume'] = sm.road_links[('volume', 'pt')]

In [ ]:
df = sm.links
df = df.loc[df['volume'] > 0]

In [ ]:
if manual:
    import matplotlib.pyplot as plt
    from syspy.syspy_utils import data_visualization as dv
    spectral = list(reversed(
        ['#9e0142','#d53e4f','#f46d43','#fdae61','#fee08b',
         '#e6f598','#abdda4','#66c2a5','#3288bd','#5e4fa2']))

    fig, ax = plt.subplots(figsize=[17, 15])
    lv = (5000,3000, 2000, 1000)
    df = sm.road_links
    df['volume'] = df[('volume', 'car')]
    df = df.dropna(subset=['volume'])

    plot = dv.bandwidth(
        df, value_column='volume', power=0.5, max_value=lv[0], legend_values=lv,
        legend_length=0.5, label_column='label', max_linewidth_meters=100,
        cmap=spectral,
        label_kwargs={'size':14, 'color':'black'}, ax=ax
    )

    lv = (12000,10000, 7000,5000, 2000, 1000)
    fig, ax = plt.subplots(figsize=[17, 15])
    infra_links = sm.links.loc[sm.links.road_length.isnull()]

    sm.road_links['volume'] = sm.road_links[('volume', 'pt')]
    df = pd.concat([infra_links, sm.road_links])
    df['volume'] = df['volume'].fillna(0)
    df = df.loc[df['volume'] > 0]

    plot = dv.bandwidth(
        df, value_column='volume', power=0.5, max_value=lv[0],legend_values=lv,
        legend_length=0.5, label_column='label', max_linewidth_meters=200,
        #geographical_bounds=list(raster_montreal.bounds),
        cmap=spectral,
        label_kwargs={'size':14, 'color':'black'}, ax=ax
    )

In [ ]:
sm.pt_los = sm.los.loc[sm.los['route_type'] != 'car']
sm.to_zip(quetzal_path + 'model/' +scenario + '/assigned.zip', omitted_attributes=['pt_los', 'car_los'])